In [ ]:
!pip install open3d
import os
import glob
import open3d as o3d
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

2.1


In [49]:
pcd_files = glob.glob('/content/drive/MyDrive/lidar/lidar_scans/*.pcd')

lidar_normals = []
lidar_offsets = []
lidar_points = []

for idx, pcd in enumerate(pcd_files):
  pcd = o3d.io.read_point_cloud(pcd)
  points = np.asarray(pcd.points)
  lidar_points.append(points)

  centroid = np.mean(points, axis=0)

  shifted_points = points - centroid

  U, s, Vt = np.linalg.svd(shifted_points)

  lidar_normals.append(Vt[-1])
  lidar_offsets.append(-np.dot(Vt[-1], centroid))

lidar_offsets = np.array(lidar_offsets)
lidar_normals = np.array(lidar_normals)

2.2


In [ ]:
camera_normal_folder = glob.glob('/content/drive/MyDrive/lidar/camera_parameters/*.jpeg/camera_normals.txt')
rotation_matrix_folder = glob.glob('/content/drive/MyDrive/lidar/camera_parameters/*.jpeg/rotation_matrix.txt')
translation_matrix_folder = glob.glob('/content/drive/MyDrive/lidar/camera_parameters/*.jpeg/translation_vectors.txt')

camera_normals = []
for idx, cn in enumerate(camera_normal_folder):
  nrml = []
  with open(cn, 'r') as f:
    for line in f:
      nrml.append(np.asarray(line[:-1], dtype='float64'))
  camera_normals.append(nrml)
camera_normals = np.array(camera_normals)

rotation_matrix = []
for idx, cn in enumerate(rotation_matrix_folder):
  nrml = []
  with open(cn, 'r') as f:
    for line in f:
      nrml.append(np.asarray(line[:-1].split(), dtype='float64'))
  rotation_matrix.append(nrml)

translation_vector = []
for idx, cn in enumerate(translation_matrix_folder):
  nrml = []
  with open(cn, 'r') as f:
    for line in f:
      nrml.append(np.asarray(line[:-1], dtype='float64'))
  translation_vector.append(nrml)

In [ ]:
world_points = np.array([1,1,0])

camera_offsets = []
for i in range(len(rotation_matrix)):
  mul = np.matmul(world_points, np.array(rotation_matrix[i]))
  cmr_pt = np.array(np.add(mul, np.array(translation_vector[i])))
  
  offst = np.matmul(cmr_pt,camera_normals[i])
  camera_offsets.append(offst)
  
camera_offsets = np.array(camera_offsets)

2.3


In [ ]:
cmr = np.array(camera_normals)
inv = np.linalg.inv(np.matmul(cmr.T,cmr))
inv_mul_transpose = np.matmul(inv,cmr.T)
offst_diff = np.array(camera_offsets)-np.array(lidar_offsets)
t1 = np.matmul(inv_mul_transpose, offst_diff)

prod = np.matmul(camera_normals.T, lidar_normals)
U, s, Vt = np.linalg.svd(prod)
R1 = np.matmul(Vt, U.T)

1.4


In [55]:
CHECKERBOARD = (6,8)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
objpoints = []
imgpoints = []

objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

calibration_images = glob.glob('/content/drive/MyDrive/lidar/camera_images/*.jpeg')
print(len(calibration_images))

for idx, fname in enumerate(calibration_images):
  img = cv2.imread(fname)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  ret, corners = cv2.findChessboardCorners(gray, (CHECKERBOARD[0], CHECKERBOARD[1]), None)
  
  if ret == True:
      objpoints.append(objp)
      imgpoints.append(corners)
      
      # cv2.waitKey(500)  

objpoints = np.array(objpoints)
imgpoints = np.array(imgpoints)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

25


In [56]:
np.array(mtx).shape

(3, 3)

2.4


In [58]:
lidar_points = np.array(lidar_points)
camera_trnsf_points = []

for i in range(len(lidar_points)):

  tmp = []
  for pt in lidar_points[i]:
    arr = np.matmul(np.array(pt),mtx)
    tmp.append(arr)
  camera_trnsf_points.append(tmp)
